In [1]:
import torch
from torch import nn

In [2]:
class MLP(nn.Module):
    def __init__(self,**kwargs):
        super(MLP,self).__init__(**kwargs)
        self.hidden=nn.Linear(784,256) #隐藏层
        self.act=nn.ReLU()
        self.output=nn.Linear(256,10) #输出层
        
    def forward(self,x):
        a=self.act(self.hidden(x))
        return self.output(a)

In [3]:
X=torch.rand(2,784)
net=MLP()
print(net)
net(X)

MLP(
  (hidden): Linear(in_features=784, out_features=256, bias=True)
  (act): ReLU()
  (output): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.0682, -0.1003, -0.1434, -0.0893, -0.0592,  0.1522, -0.1157,  0.2100,
          0.1044, -0.0775],
        [-0.1100, -0.0498, -0.0563, -0.0125,  0.0095, -0.0157, -0.1008,  0.1075,
          0.0366, -0.0704]], grad_fn=<AddmmBackward>)

In [4]:
class MySequential(nn.Module):
    from collections import OrderedDict
    def __init__(self,*args):
        super(MySequential,self).__init__()
        if len(args)==1 and isinstance(args[0],OrderedDict):
            for key,module in args[0].items():
                self.add_module(key,module)
        else:
            for idx,module in enumerate(args):
                self.add_module(str(idx),module)
    def forward(self,input):
        for module in self._modules.values():
            input=module(input)
        return input

In [5]:
net=MySequential(
    nn.Linear(784,256),
    nn.ReLU(),
    nn.Linear(256,10),
)

In [6]:
print(net)
net(X)

MySequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


tensor([[-0.1724,  0.1396, -0.1365,  0.0698,  0.2365,  0.0544, -0.2704,  0.0338,
          0.0088,  0.0474],
        [-0.2342,  0.2277, -0.1147, -0.2756,  0.2801,  0.1294, -0.1871,  0.1507,
         -0.0284,  0.1048]], grad_fn=<AddmmBackward>)

In [7]:
net=nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
net.append(nn.Linear(256,10))
print(net[-1])
print(net)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [8]:
class MyModule(nn.Module):
    def __init__(self):
        super(MyModule,self).__init__()
        self.linears=nn.ModuleList([nn.Linear(10,10) for i in range(10)])
        
    def forward(self,x):
        for i,l in enumerate(self.linears):
            x=self.linears[i//2](x)+l(x)
        return x
    

In [10]:
class Module_ModuleList(nn.Module):
    def __init__(self):
        super(Module_ModuleList,self).__init__()
        self.linears=nn.ModuleList([nn.Linear(10,10)])
        
class Module_List(nn.Module):
    def __init__(self):
        super(Module_List,self).__init__()
        self.linears=[nn.Linear(10,10)]

In [11]:
net1=Module_ModuleList()
net2=Module_List()

print("net1:")
for p in net1.parameters():
    print(p.size())
    
print("net2:")
for p in net2.parameters():
    print(p.size())

net1:
torch.Size([10, 10])
torch.Size([10])
net2:


In [12]:
net=nn.ModuleDict({
        'linear':nn.Linear(184,256),
        'act':nn.ReLU(),
})
net['output']=nn.Linear(256,10)
print(net['linear'])
print(net.output)
print(net)

Linear(in_features=184, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)
ModuleDict(
  (act): ReLU()
  (linear): Linear(in_features=184, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)


In [13]:
class FancyMLP(nn.Module):
    def __init__(self,**kwargs):
        super(FancyMLP,self).__init__(**kwargs)
        
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)
    
    def forward(self,x):
        x=self.linear(x)
        x=nn.functional.relu(torch.mm(x,self.rand_weight.data)+1)
        
        x=self.linear(x)
        
        while x.norm().item()>1:
            x/=2
        if x.norm().item()<0.8:
            x*=10
        return x.sum()

In [14]:
X=torch.rand(2,20)
net=FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(-1.3694, grad_fn=<SumBackward0>)

In [15]:
class NestMLP(nn.Module):
    def __init__(self,**kwargs):
        super(NestMLP,self).__init__(**kwargs)
        self.net=nn.Sequential(nn.Linear(40,30),nn.ReLU())
        
    def forward(self,x):
        return self.net(x)

net=nn.Sequential(NestMLP(),nn.Linear(30,20),FancyMLP())

X=torch.randn(2,40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(-16.3599, grad_fn=<SumBackward0>)